In [1]:
import dask_cudf
import cupy as cp
import argparse
import time
import gcsfs
import dask_cudf
import os, json
import subprocess

In [2]:

from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait
from dask.utils import parse_bytes

cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),)
client = Client(cluster)
client


Client Scheduler: tcp://127.0.0.1:37863 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 15.77 GB


In [3]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait

cluster = LocalCUDACluster()
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:35007 Dashboard: http://127.0.0.1:43169/status,Cluster Workers: 1 Cores: 1 Memory: 15.77 GB


In [3]:
%%time
colnames = ['label'] + ['feature%02d' % i for i in range(1, 29)]
train_dir='gs://nvidiadask/higgs2/*.csv'
df = dask_cudf.read_csv(train_dir, header=None, names=colnames, chunksize="500 MiB")

CPU times: user 1.04 s, sys: 440 ms, total: 1.48 s
Wall time: 4.79 s


In [4]:
%%time
df["key"] = df.feature02.round()
group_means = df.groupby("key").mean().persist()
wait(group_means);

group_means.head()

MemoryError: std::bad_alloc: CUDA error at: ../include/rmm/mr/device/cuda_memory_resource.hpp:68: cudaErrorMemoryAllocation out of memory

In [ ]:
from xgboost import dask as dxgb
from xgboost.dask import DaskDMatrix

import xgboost as xgb

X = df[df.columns.difference(['label'])]
y = df['label']
    
# `DaskDeviceQuantileDMatrix` is used instead of `DaskDMatrix`, be careful
   # that it can not be used for anything else than training.
start_time = time.time()
dtrain = dxgb.DaskDeviceQuantileDMatrix(client, X, y)
print("[INFO]: ------ DMatrix is formed in {} seconds ---".format((time.time() - start_time)))
   
del df
del X
del y

start_time = time.time()
output = xgb.dask.train(client,
                     { 'verbosity': 2,
                         'learning_rate': 0.1,
                          'max_depth': 8,
                          'objective': 'reg:squarederror',
                          'subsample': 0.5,
                          'gamma': 0.9,
                          'verbose_eval': True,
                          'tree_method':'gpu_hist',
                          #'nthread':1
                        },
                        dtrain,
                        num_boost_round=100, evals=[(dtrain, 'train')])
print("[INFO]: ------ Training is completed in {} seconds ---".format((time.time() - start_time)))
    
history = output['history']
print('[INFO]: ------ Training evaluation history:', history)

# Deploy model

In [ ]:
MODEL_NAME = "XGBoost_Dask"
VERSION_NAME = "xgboost_dask_{}".format(int(time.time()))

In [ ]:
!gcloud ai-platform models create $MODEL_NAME --regions us-central1